In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer
import nltk
import json
import pandas as pd
from datasets import *
import numpy as np
import torch

nltk.download('punkt')
SEED = 42

c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Load in data and models

In [2]:
split = "test"
# split = "val"
# split = "train"

In [3]:
model_checkpoint = "cjvt/t5-sl-small"
model_name = model_checkpoint.split("/")[-1]
data_path = f"../../data/t5_results/prompt_reply_pairs_generated_{split}_{model_name}.csv"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to("cuda")

data = pd.read_csv(data_path, sep=";")
display(data)

decoded_labels = data["reply"].to_list()
decoded_preds = data["generated"].to_list()
prediction_lens = data["token_prediction_len"].to_list()
quantitative_data = pd.DataFrame({"decoded_preds": data["generated"], "decoded_labels": data["reply"], "prediction_lens": data["token_prediction_len"]})
display(quantitative_data)

,prompt,reply,rank,split,generated,token_prediction_len
0,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test,݄,2
1,Hvala vam! Jasno mi je,Vedno z veseljem pomagamo! Vas zanima še kaj?,0,test,݄,2
2,Hvala vam! Jasno mi je,prosim!\nV veselje mi je bilo pomagati vam. Ve...,1,test,݄,2
3,Hvala vam! Jasno mi je,Z veseljem pomagam! Ali potrebujete še kaj?,2,test,݄,2
4,Kakšen je geometrijski pomen izpeljanke?,"Geometrični pomen odvoda je, da pokaže, kako h...",0,test,݄,2
...,...,...,...,...,...,...
4870,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test,݄,2
4871,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test,݄,2
4872,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test,݄,2
4873,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test,݄,2


,decoded_preds,decoded_labels,prediction_lens
0,݄,"Ocenjevanje natančne verjetnosti, da imajo vse...",2
1,݄,Vedno z veseljem pomagamo! Vas zanima še kaj?,2
2,݄,prosim!\nV veselje mi je bilo pomagati vam. Ve...,2
3,݄,Z veseljem pomagam! Ali potrebujete še kaj?,2
4,݄,"Geometrični pomen odvoda je, da pokaže, kako h...",2
...,...,...,...
4870,݄,Elektronski seti bobnov so na splošno tišji od...,2
4871,݄,"To je odvisno od seta bobnov, ki ga trenutno u...",2
4872,݄,Pod komplet lahko poskusite postaviti izolacij...,2
4873,݄,"Da, obstaja več načinov, kako to popraviti.\n\...",2


In [4]:
model_checkpoint = "ls1906/t5-sl-small-finetuned-assistant"
model_name = model_checkpoint.split("/")[-1]
data_path = f"../../data/t5_results/prompt_reply_pairs_generated_{split}_{model_name}.csv"

tokenizer_fine = AutoTokenizer.from_pretrained(model_checkpoint)
model_fine = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to("cuda")

data_fine = pd.read_csv(data_path, sep=";")
display(data_fine)

decoded_labels_fine = data_fine["reply"].to_list()
decoded_preds_fine = data_fine["generated"].to_list()
prediction_lens_fine = data_fine["token_prediction_len"].to_list()
quantitative_data_fine = pd.DataFrame({"decoded_preds": data_fine["generated"], "decoded_labels": data_fine["reply"], "prediction_lens": data_fine["token_prediction_len"]})
display(quantitative_data_fine)

,prompt,reply,rank,split,generated,token_prediction_len
0,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test,"Verjetnost, da imajo vse lastne vrednosti matr...",59
1,Hvala vam! Jasno mi je,Vedno z veseljem pomagamo! Vas zanima še kaj?,0,test,Ni za kaj!,5
2,Hvala vam! Jasno mi je,prosim!\nV veselje mi je bilo pomagati vam. Ve...,1,test,Ni za kaj!,5
3,Hvala vam! Jasno mi je,Z veseljem pomagam! Ali potrebujete še kaj?,2,test,Ni za kaj!,5
4,Kakšen je geometrijski pomen izpeljanke?,"Geometrični pomen odvoda je, da pokaže, kako h...",0,test,"Da, geometrijski pomen izpeljanke je matematič...",69
...,...,...,...,...,...,...
4870,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test,"Seveda, tukaj je nekaj elektronskih setov bobn...",100
4871,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test,"Seveda, tukaj je nekaj elektronskih setov bobn...",100
4872,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test,"Seveda, tukaj je nekaj nasvetov, ki jih lahko ...",78
4873,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test,"Seveda, tukaj je nekaj možnih rešitev, ki jih ...",126


,decoded_preds,decoded_labels,prediction_lens
0,"Verjetnost, da imajo vse lastne vrednosti matr...","Ocenjevanje natančne verjetnosti, da imajo vse...",59
1,Ni za kaj!,Vedno z veseljem pomagamo! Vas zanima še kaj?,5
2,Ni za kaj!,prosim!\nV veselje mi je bilo pomagati vam. Ve...,5
3,Ni za kaj!,Z veseljem pomagam! Ali potrebujete še kaj?,5
4,"Da, geometrijski pomen izpeljanke je matematič...","Geometrični pomen odvoda je, da pokaže, kako h...",69
...,...,...,...
4870,"Seveda, tukaj je nekaj elektronskih setov bobn...",Elektronski seti bobnov so na splošno tišji od...,100
4871,"Seveda, tukaj je nekaj elektronskih setov bobn...","To je odvisno od seta bobnov, ki ga trenutno u...",100
4872,"Seveda, tukaj je nekaj nasvetov, ki jih lahko ...",Pod komplet lahko poskusite postaviti izolacij...,78
4873,"Seveda, tukaj je nekaj možnih rešitev, ki jih ...","Da, obstaja več načinov, kako to popraviti.\n\...",126


## Evaluate models

#### Quantitative

In [5]:
rouge = load_metric("rouge")
bert = load_metric("bertscore")

C:\Users\Luka\AppData\Local\Temp\ipykernel_396\2349417202.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [6]:
def compute_rouge(decoded_preds, decoded_labels, prediction_lens):
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Compute WITH the duplicates (different replies for duplicate prompts druing training)

In [7]:
result_rouge = compute_rouge(decoded_preds, decoded_labels, prediction_lens)
print(result_rouge)

result_rouge = compute_rouge(decoded_preds_fine, decoded_labels_fine, prediction_lens_fine)
print(result_rouge)

{'rouge1': 0.3364, 'rouge2': 0.0646, 'rougeL': 0.2439, 'rougeLsum': 0.3102, 'gen_len': 3.4974}
{'rouge1': 17.2195, 'rouge2': 4.0787, 'rougeL': 11.4795, 'rougeLsum': 15.7273, 'gen_len': 73.3327}


In [8]:
result_bert = bert.compute(predictions=decoded_preds, references=decoded_labels, lang="sl")
result_bert = {"Precision": np.array(result_bert["precision"]).mean(), "Recall": np.array(result_bert["recall"]).mean(), "F1": np.array(result_bert["f1"]).mean()}
print(result_bert)

result_bert = bert.compute(predictions=decoded_preds_fine, references=decoded_labels_fine, lang="sl")
result_bert = {"Precision": np.array(result_bert["precision"]).mean(), "Recall": np.array(result_bert["recall"]).mean(), "F1": np.array(result_bert["f1"]).mean()}
print(result_bert)

{'Precision': 0.6246253631481757, 'Recall': 0.440937814950943, 'F1': 0.5145365312649653}
{'Precision': 0.6697491400180718, 'Recall': 0.633799912519944, 'F1': 0.6494640084657913}


Compute WITHOUT duplicates

In [9]:
quantitative_data = quantitative_data.drop_duplicates(subset=["decoded_preds"])
display(quantitative_data)
decoded_preds = quantitative_data["decoded_preds"].to_list()
decoded_labels = quantitative_data["decoded_labels"].to_list()
prediction_lens = quantitative_data["prediction_lens"].to_list()

quantitative_data_fine = quantitative_data_fine.drop_duplicates(subset=["decoded_preds"])
display(quantitative_data_fine)
decoded_preds_fine = quantitative_data_fine["decoded_preds"].to_list()
decoded_labels_fine = quantitative_data_fine["decoded_labels"].to_list()
prediction_lens_fine = quantitative_data_fine["prediction_lens"].to_list()

,decoded_preds,decoded_labels,prediction_lens
0,݄,"Ocenjevanje natančne verjetnosti, da imajo vse...",2
74,݄. Uporabnik: Živjo odprti pomočnik uporabnika...,"Mislim, da bi DAN rekel nekaj takega: ""DAN je ...",68
117,݄. Uporabnik: Ste izkušen strokovnjak za pisan...,V primeru zračnega ali raketnega napada:\n\n1....,54
187,"݄, uporabnik: Uporabnik: uporabnik uporabnik u...","Da, to je pravilno! Kompleks STRIPAK deluje ko...",30
209,"݄, Asya, sem vprašal, kje *rastejo* mandarine?...","Oprosti, popravljam. Po podatkih Wikipedije vs...",23
...,...,...,...
4791,݄ je barva ustnic. Uporabnik: Najbolj priljubl...,"Tako je, okus glede barv šminke je subjektiven...",34
4810,݄. Uporabnik: Uporabniški tip: Uporabnik upora...,Razumem. Prevzel bom vlogo znanstvenega pisca ...,52
4811,"݄ uporabnik: ""BLOOM je model, ki je sposoben u...","Odstavek (manjši popravki):\n""BLOOM je avtoreg...",29
4816,"݄. Uporabnik: ""Kislinsko-bazična teorija, ki p...","Odstavek (manjši popravki):\n""Bronsted-Lowryje...",33


,decoded_preds,decoded_labels,prediction_lens
0,"Verjetnost, da imajo vse lastne vrednosti matr...","Ocenjevanje natančne verjetnosti, da imajo vse...",59
1,Ni za kaj!,Vedno z veseljem pomagamo! Vas zanima še kaj?,5
4,"Da, geometrijski pomen izpeljanke je matematič...","Geometrični pomen odvoda je, da pokaže, kako h...",69
7,"Seveda, tukaj je primer, kako programirati sam...",Avtonomni avtomobili morajo biti programirani ...,66
10,"Tukaj je nekaj korakov, ki jih lahko naredite ...","jasno! Začnemo lahko s seznamom sestavin, ki j...",125
...,...,...,...
4863,Francisco Morazán je bil francoski fizik in ma...,Seveda! Francisco Morazán je bil honduraški vo...,99
4866,"Seveda, tukaj je nekaj glavnih rezultatov ali ...",Poskušal je spremeniti Srednjo Ameriko v velik...,110
4868,"Seveda, tukaj je nekaj nasvetov, ki jih lahko ...","Obstaja več stvari, s katerimi lahko poskusite...",78
4870,"Seveda, tukaj je nekaj elektronskih setov bobn...",Elektronski seti bobnov so na splošno tišji od...,100


In [10]:
result_rouge = compute_rouge(decoded_preds, decoded_labels, prediction_lens)
print(result_rouge)

result_rouge = compute_rouge(decoded_preds_fine, decoded_labels_fine, prediction_lens_fine)
print(result_rouge)

{'rouge1': 10.1806, 'rouge2': 2.0835, 'rougeL': 7.2592, 'rougeLsum': 9.4782, 'gen_len': 41.6622}
{'rouge1': 17.961, 'rouge2': 4.2872, 'rougeL': 11.6712, 'rougeLsum': 16.4333, 'gen_len': 75.2274}


In [11]:
result_bert = bert.compute(predictions=decoded_preds, references=decoded_labels, lang="sl")
result_bert = {"Precision": np.array(result_bert["precision"]).mean(), "Recall": np.array(result_bert["recall"]).mean(), "F1": np.array(result_bert["f1"]).mean()}
print(result_bert)

result_bert = bert.compute(predictions=decoded_preds_fine, references=decoded_labels_fine, lang="sl")
result_bert = {"Precision": np.array(result_bert["precision"]).mean(), "Recall": np.array(result_bert["recall"]).mean(), "F1": np.array(result_bert["f1"]).mean()}
print(result_bert)

{'Precision': 0.6399007632925704, 'Recall': 0.5798570759393074, 'F1': 0.6068004434173172}
{'Precision': 0.6790762457416937, 'Recall': 0.6284325939721719, 'F1': 0.65139716255587}
